<a target="_blank" href="https://colab.research.google.com/github/greatakela/ChatBot/blob/main/Notebooks/GNLP_HW1-bi_encoder_model_train.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
!pip install -Uq sentence-transformers datasets transformers wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bagor123 (greatakela) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Обучение модели энкодера

Для ранжирования и отбора кандидатов обучим модель энкодера на основе библиотеки sentence transformers

In [4]:
import torch
import gc
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample
from sentence_transformers import SentenceTransformer, models, losses, evaluation
import datasets
from datasets import Dataset

In [7]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [6]:
model_name = "distilroberta-base"
#model_name = "sentence-transformers/LaBSE"
#model_name = "sentence-transformers/all-mpnet-base-v2"
max_seq_length = 128
num_epochs = 3
train_batch_size = 8

Для ранжирования реплик и подбора наилучших ответов используем обучение с функцией потерь Triplet Loss - минимизация расстояния от вопроса и опорных контекстных фраз до "правильных" ответов и максимизация расстояния до "неправильных" ответов.


In [8]:
## Собираем модель:
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Загружаем данные, которые уже обработаны до триплетов:
- ANCHOR - комбинация контекста и вопросов
- ANSWER - ответ на вопрос из скрипта
- WRONG_ANSWER - случайно подобранный ответ
- INTENT_TAG - тип реплики


In [9]:
spock_lines = pd.read_pickle("/content/drive/MyDrive/GNLP/HW1/spock_lines.pkl")
spock_lines

,QUESTION,ANSWER,CONTEXT,ANCHOR,WRONG_ANSWER,INTENT_TAG
0,,Check the circuit.,,,Does it matter?,Statement-non-opinion
1,"All operating, sir.",It can't be the screen then. Definitely someth...,Check the circuit.,"Check the circuit.[SEP]All operating, sir.",He was in the Navy not the Marines.,Statement-non-opinion
2,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"Steady as we go. It's a radio wave, sir. We're...","Steady as we go. It's a radio wave, sir. We're...","This is not an act, I don't care if--",Statement-non-opinion
3,It would take that long for a radio beam to tr...,Records show the Talos group has never been ex...,They were keyed to cause interference and attr...,They were keyed to cause interference and attr...,[Puts his hand on her shoulder.] I'm proud of...,Statement-opinion
4,If they survived the crash.,"We aren't going to go, to be certain?",We've no ships or Earth colonies that far out....,We've no ships or Earth colonies that far out....,You're lousy with faces.,Statement-non-opinion
...,...,...,...,...,...,...
5752,,Alter circuit A.,,,The heart shows no sign of inflammation or...,Statement-non-opinion
5753,I'd give real money if he'd shut up.,Plate please.,...discuss the report on Phase two... Alter ci...,...discuss the report on Phase two... Alter ci...,"Hello yes, doctor House, its Esteban Hernande...",Statement-non-opinion
5754,,Key please.,,,"Got it, we're all idiots, what's your theory?",Statement-non-opinion
5755,Arrest those men!,Arrest yourself.,Out of the way! Out of the way! Mister Preside...,Out of the way! Out of the way! Mister Preside...,Your personal life is none of my business.,Quotation


In [10]:
spock_lines['ANCHOR'][5755]

'Out of the way! Out of the way! Mister President! ...Mister President! Kirk, ...Enterprise. Protect the Chancellor.[SEP]Arrest those men!'

In [11]:
examples = []

for index, row in spock_lines.iterrows():
    examples.append(InputExample(texts=[row["ANCHOR"], row["ANSWER"], row['WRONG_ANSWER']]))
train_examples, valid_examples = train_test_split(examples, test_size=0.15)

In [12]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
valid_dataloader = DataLoader(valid_examples, shuffle=False, batch_size=train_batch_size)

TripletEvaluator проверяет, если расстояние(ANCHOR, ANSWER) меньше чем расстояние (ANCHOR, WRONG_ANSWER).

In [13]:
train_loss = losses.TripletLoss(model=model)

In [14]:
evaluator_enc = evaluation.TripletEvaluator.from_input_examples(train_examples, name = 'evaluator_enc', batch_size=8, write_csv=True)

In [15]:
%%time
model.evaluate(evaluator_enc)

CPU times: user 19 s, sys: 2.71 s, total: 21.7 s
Wall time: 14.6 s


{'evaluator_enc_cosine_accuracy': 0.5865522027015686}

In [16]:
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data

In [17]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    scheduler="warmupcosine",
    save_best_model=True,
    evaluator=evaluator_enc,
    evaluation_steps=300,
    output_path='/content/drive/MyDrive/GNLP/HW1/gnlp_hw1_encoder/'
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Evaluator Enc Cosine Accuracy
300,No log,No log,0.987533
600,1.085000,No log,0.993460
612,1.085000,No log,0.993664
900,1.085000,No log,0.996730
1200,0.157300,No log,0.997956
1224,0.157300,No log,0.997956
1500,0.073300,No log,0.998978
1800,0.073300,No log,0.998978
1836,0.073300,No log,0.998978


In [18]:
evaluator_val = evaluation.TripletEvaluator.from_input_examples(valid_examples, name="evaluator_val", batch_size=8, write_csv=True)
model.evaluate(evaluator_val)

{'evaluator_val_cosine_accuracy': 0.9872685074806213}

In [19]:
model.save('/content/drive/MyDrive/GNLP/HW1/gnlp_hw1_encoder')

In [20]:
model.push_to_hub("greatakela/gnlp_hw1_encoder", exist_ok=True)

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

'https://huggingface.co/greatakela/gnlp_hw1_encoder/commit/25b27e5ff23e13f7e8eb4ac8187f7cc294df6773'

In [21]:
model_enc = SentenceTransformer("/content/drive/MyDrive/GNLP/HW1/gnlp_hw1_encoder")

In [22]:
def encode(texts, model, intent, contexts=None, do_norm=True):
    """function to encode texts for cosine similarity search"""

    question_vectors = model.encode(texts)
    context_vectors = model.encode("".join(contexts))
    intent_vectors = model.encode(intent)

    return np.concatenate(
        [
            np.asarray(context_vectors),
            np.asarray(question_vectors),
            np.asarray(intent_vectors),
        ],
        axis=-1,
    )

In [23]:
def encode_df_save(model):
    """this functions vectorizes reworked scripts and loads them to
    pickle file to be used as retrieval base for ranking script"""

    scripts_reopened = pd.read_pickle("/content/drive/MyDrive/GNLP/HW1/spock_lines.pkl")
    vect_data = []
    for index, row in scripts_reopened.iterrows():
        vect = encode(
            texts=row["QUESTION"],
            model=model,
            intent=row["INTENT_TAG"],
            contexts=row["CONTEXT"],
        )
        vect_data.append(vect)
    with open("/content/drive/MyDrive/GNLP/HW1/spock_lines_vectorized.pkl", "wb") as f:
        pickle.dump(vect_data, f)

In [24]:
encode_df_save(model_enc)